In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil

try:
  import google.colab
  os.system("git clone https://github.com/matt-nann/AuthenticCursor.git")
  try:
    shutil.copytree("AuthenticCursor/src", "src")
  except:
    shutil.rmtree("src")
    shutil.copytree("AuthenticCursor/src", "src")
  try:
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  except:
    shutil.rmtree("requirementsGAN.txt")
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  # remove conflicting dependencies with google colab preinstalled libraries
  with open("requirementsGAN.txt", "r") as f:
    lines = f.readlines()
    with open("requirementsGAN.txt", "w") as f:
      for line in lines:
        if "numpy" not in line and 'pillow' not in line:
          f.write(line)
  os.system("pip install -r requirementsGAN.txt")
  shutil.rmtree("AuthenticCursor")
  # installing and logging into weights and biases
  os.system("pip install wandb")
  os.system("wandb login")
except Exception as e:
  print(e)

import torch
import wandb # will be prompted for API key in google colab

In [ ]:
from src.mouseGAN.dataProcessing import MouseGAN_Data
from src.mouseGAN.dataset import getDataloader, visuallyVertifyDataloader

USE_FAKE_DATA = True
SAVE_FAKE_DATA = False
RELOAD_FAKE_DATA = True
TRAIN_TEST_SPLIT = 0.8
dataset = MouseGAN_Data(USE_FAKE_DATA=USE_FAKE_DATA, TRAIN_TEST_SPLIT=TRAIN_TEST_SPLIT, 
                        equal_length=False)

SAMPLES = 40000
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if USE_FAKE_DATA:
    if RELOAD_FAKE_DATA:
        # dataset.createFakeWindMouseDataset(save=SAVE_FAKE_DATA, samples=SAMPLES,
        #                                 low_radius = 200, high_radius = 1000,
        #                                 max_width = 300, min_width = 25,
        #                                 max_height = 300, min_height = 25,)
        dataset.createFakeWindMouseDataset(save=SAVE_FAKE_DATA, samples=SAMPLES,
                                        low_radius = 65, high_radius = 200,
                                        max_width = 60, min_width = 50,
                                        max_height = 60, min_height = 50,)
    else:
        dataset.loadFakeWindMouseData()
else:
    df_moves, df_trajectory = dataset.collectRawMouseTrajectories()

In [ ]:
import time
s_time = time.time()
train_trajs, train_targets, test_trajs, test_targets = dataset.processMouseData(SHOW_ALL=False)
print(f"Time to process data: {time.time() - s_time} seconds")

In [34]:
from src.mouseGAN.model_config import Config, LR_SCHEDULERS, LOSS_FUNC, \
    C_MiniBatchDisc, C_Discriminator, C_Generator, C_EMA_Plateua_Sch, \
    C_Step_Sch, C_LossGap_Sch
from src.mouseGAN.models import MouseGAN
from src.mouseGAN.experimentTracker import initialize_wandb

# IN_COLAB = True
LOAD_PRETRAINED = False
BATCH_SIZE = 256
num_epochs = 1000
num_feats = train_trajs[0].shape[1]
latent_dim = 100
num_target_feats = 4 # width, height, start_x, start_y
numBatches = len(train_trajs)//BATCH_SIZE
MAX_SEQ_LEN = max([len(traj) for traj in train_trajs + test_trajs])

D_config = C_Discriminator(lr=0.005, bidirectional=True, hidden_units=128, 
                            num_lstm_layers=2, 
                            useEndDeviationLoss=True,
                            gradient_maxNorm = 1.0,)
G_config = C_Generator(lr=0.0005, hidden_units=128, num_lstm_layers=2, drop_prob=0.4,
                layer_normalization = True,
                residual_connections = True,
                gradient_maxNorm = 1.0,
                useSeqLengthLoss=False,
                lengthLossWeight = 0.25,
                useOutsideTargetLoss=True,
                outsideTargetLossWeight = 1,
                usePathLengthLoss=True,
                pathLengthLossWeight = 0.25)

# D_sch_config = C_Step_Sch(2, 0.5)
D_sch_config = C_LossGap_Sch(cooldown=int(numBatches/2), lr_shrinkMin=0.1, lr_growthMax=2.0, 
                            discLossDecay=0.8, lr_max = D_config.lr, lr_min = 0.00005, restart_after=int(numBatches/2))
# G_sch_config = C_Step_Sch(2, 0.5)
# G_sch_config = C_EMA_Plateua_Sch(patience=BATCH_SIZE, cooldown=int(BATCH_SIZE/8), factor=0.5, ema_alpha=0.4)

config = Config(num_epochs, BATCH_SIZE, num_feats, latent_dim, num_target_feats, MAX_SEQ_LEN,
                discriminator=D_config, generator=G_config, 
                D_lr_scheduler=D_sch_config, #G_lr_scheduler=G_sch_config,
                locationMSELoss = False)

## verifying the mean trajectory is centered around zero (even class distribution)
# dataset.plotMeanPath()
trainLoader = getDataloader(train_trajs, train_targets, config.BATCH_SIZE)
testLoader = getDataloader(test_trajs, test_targets, config.BATCH_SIZE)

visuallyVertifyDataloader(trainLoader, dataset, showNumBatches=1)

if IN_COLAB:
    run = initialize_wandb(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan = MouseGAN(dataset, trainLoader, testLoader, device, config, IN_COLAB=IN_COLAB, verbose=True, printBatch=True)
if LOAD_PRETRAINED:
    
    gan.loadPretrained(startingEpoch='final')

print(gan.discriminator)
print(gan.generator)

# gan.find_learning_rates_for_GAN()
# gan.train(modelSaveInterval=3, catchErrors=False)
if IN_COLAB:
    wandb.finish()

{'cooldown': 32, 'type': <LR_SCHEDULERS.LOSS_GAP_AWARE: 'loss_gap_aware_lr_scheduler'>, 'ideal_loss': 0.5, 'loss_min': 0.05, 'loss_max': 0.05, 'lr_shrinkMin': 0.1, 'lr_growthMax': 2.0, 'discLossDecay': 0.8, 'lr_max': 0.001, 'lr_min': 1e-09}
Discriminator(
  (miniBatch_d): MinibatchDiscrimination()
  (lstm_d): LSTM(11, 128, num_layers=2, batch_first=True, bidirectional=True)
  (score_layer_d): Linear(in_features=256, out_features=1, bias=True)
  (endLoc_layer_d): Linear(in_features=256, out_features=2, bias=True)
)
Generator(
  (fc_input_g): Linear(in_features=106, out_features=128, bias=True)
  (fc_sequenceLength): Linear(in_features=128, out_features=1, bias=True)
  (lstm_cells_g): ModuleList(
    (0-1): 2 x LSTMCell(128, 128)
  )
  (layer_norms_g): ModuleList(
    (0-1): 2 x LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc_output_g): Linear(in_features=128, out_features=2, bias=Tru

/Users/mnann/Documents/Code/AuthenticCursor/venvDev/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning:

torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.



In [ ]:
gan.visualTrainingVerfication(samples=10)

In [ ]:
gan.train(modelSaveInterval=3, catchErrors=False)

In [ ]:
# gan.save_models('final')
gan.loadPretrained(startingEpoch=99)

In [ ]:
for epoch in ['final']:
    gan.loadPretrained(startingEpoch=epoch)
    gan.visualTrainingVerfication()